![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/competitions/miad2024-12-prediccion-precio-vehiculos).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

import seaborn as sns
import matplotlib.pyplot as plt

import joblib

import os
os.chdir('..')

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [5]:
# Visualización datos de test
dataTesting.head()

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand


In [6]:
mapeos_codificacion = {}

def var_categoricas(df):

    variables_categoricas = ['State', 'Make', 'Model']
    
    label_encoder = LabelEncoder()

    mapeos_codificacion = {}

    for i in variables_categoricas:
        df[i+'_Encod'] = label_encoder.fit_transform(df[i].astype('category')).astype(int)
        valores_originales = df[i].unique()
        valores_codificados = df[i+'_Encod'].unique()

        mapeo = {valor_original: valor_codificado for valor_original, valor_codificado in zip(valores_originales, valores_codificados)}
        mapeos_codificacion[i] = mapeo

    df.drop(variables_categoricas, axis=1, inplace=True)

    return df

X_Train = dataTraining.drop(["Price"], axis=1)
X_Train = var_categoricas(X_Train)
y_Train = dataTraining['Price']
X_Test = var_categoricas(dataTesting)

mapeos_codificacion = {}

# Inicializar el codificador de etiquetas
label_encoder = LabelEncoder()
variables_categoricas = ['State', 'Make', 'Model']

# Iterar sobre cada variable categórica
for columna in variables_categoricas:
    # Codificar los valores y almacenarlos en una nueva columna
    dataTraining[columna+'_Encod'] = label_encoder.fit_transform(dataTraining[columna].astype('category')).astype(int)
    
    # Obtener los valores únicos de la variable original
    valores_originales = dataTraining[columna].unique()
    
    # Obtener los valores únicos codificados
    valores_codificados = dataTraining[columna+'_Encod'].unique()
    
    # Crear un diccionario de mapeo para esta variable categórica
    mapeo = {valor_original: valor_codificado for valor_original, valor_codificado in zip(valores_originales, valores_codificados)}
    
    # Agregar el diccionario de mapeo al diccionario principal
    mapeos_codificacion[columna] = mapeo

In [7]:
# Entrenamiento de los modelos

xgb = BaggingRegressor(XGBRegressor(), n_estimators=10,random_state=1)
xgb.fit(X_Train, y_Train)

joblib.dump(xgb, 'model_deployment_PS4/xgboost_price.pkl', compress=3)


['model_deployment_PS4/xgboost_price.pkl']

In [10]:
bgrf = RandomForestRegressor(n_estimators=10, max_features=3, random_state=1, n_jobs=-1)
bgrf.fit(X_Train, y_Train)

joblib.dump(bgrf, 'model_deployment_PS4/bgg_random_forest_price.pkl', compress=3)

['model_deployment_PS4/bgg_random_forest_price.pkl']